<center><h1>Example of statistical analysis:</h1>
<h2>Search of gaussian signal in an exponentially decreasing background</h2> 
<h2>Part II</h2>

<h3>Loading the tools and function definition (again !)</h3>

<h4>Loading the tools:</h4>

In [ ]:
# Native Python modules
import os, sys, csv, string

# Scipy/Numpy modules
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
#get the plots in the notebook
%pylab inline

<h4>Functions definitions </h4>

In [ ]:
def bgFunc(alpha):
    return lambda x: 1.0/alpha*np.exp(-x/alpha)
def sigFunc(mu, sigma):
    return lambda x: 1.0/(np.sqrt(2*np.pi)*sigma)*np.exp(-np.power((x-mu),2)/(2*np.power(sigma,2)))

In [ ]:
bg=bgFunc(25)
sg=sigFunc(15,2)

<h4>generate the signal and background histograms</h4>

In [ ]:
def generateHisto(xRange,nbBins,func):
    binSize=(xRange[1]-xRange[0])/nbBins
    xBinLow=[]
    xiVal=[]
    xWidth=[]
    #first check which scale should be applied to the histogram
    histoIntegral=0
    for i in range(0,nbBins):
        xi=xRange[0]+1.0*i*binSize+1.0*binSize/2
        histoIntegral=histoIntegral+func(xi)
    #compute the scale to apply to the histo
    reScale=1./histoIntegral
    #now generate the histo
    for i in range(0,nbBins):
        xBinLow.append(xRange[0]+i*binSize)
        xi=xRange[0]+1.0*i*binSize+1.0*binSize/2
        xiVal.append(reScale*func(xi))
        xWidth.append(binSize)
    return [xBinLow, xiVal, xWidth]



In [ ]:
bgHisto_=generateHisto([0,100],100,bg)
sigHisto_=generateHisto([0,100],100,sg)
 
plt.bar(bgHisto_[0], bgHisto_[1], width=bgHisto_[2])     
plt.bar(sigHisto_[0], sigHisto_[1], width=sigHisto_[2],color='r')     

print('sumSig=',sum(sigHisto_[1]), 'sumBg=', sum(bgHisto_[1]))

In [ ]:
NbBgEvent_=5000
NbSigEvent_=70

sigHistoNormed=[x * NbSigEvent_ for x in sigHisto_[1]]
bgHistoNormed=[x * NbBgEvent_ for x in bgHisto_[1]]


sumSigBg=list(map(add,sigHistoNormed, bgHistoNormed))
plt.bar(sigHisto_[0], sumSigBg, width=sigHisto_[2],color='g')     
plt.bar(bgHisto_[0], bgHistoNormed, width=bgHisto_[2])     
plt.bar(sigHisto_[0], sigHistoNormed, width=sigHisto_[2],color='r')

<h4>loading the data </h4>

In [ ]:
dataHist=[183, 203, 189, 178, 169, 187, 156, 150, 164, 141, 153, 123, 122, 122, 117, 128, 125, 99, 91, 93, 85, 100, 91, 64, 80, 66, 69, 65, 64, 59, 64, 58, 55, 47, 52, 36, 54, 47, 53, 42, 34, 40, 45, 40, 32, 40, 24, 29, 30, 28, 19, 27, 23, 26, 23, 22, 28, 20, 13, 16, 13, 15, 15, 20, 18, 12, 13, 11, 13, 10, 9, 16, 10, 13, 6, 8, 13, 14, 4, 4, 9, 3, 10, 10, 8, 6, 11, 9, 6, 4, 6, 4, 9, 3, 5, 10, 5, 5, 5, 4]

In [ ]:
plt.bar(sigHisto_[0], dataHist, width=sigHisto_[2],color='g')  

In [ ]:
def pseudoDataFromHisto(histoSig,histoBg, NbSigEvent,NbBgEvent, mu):
    smearedHisto=[]
    sigHistoNormed=[x * NbSigEvent* mu for x in histoSig]
    bgHistoNormed=[x * NbBgEvent for x in histoBg]
    theHisto=map(add,sigHistoNormed, bgHistoNormed)
    for aBin in theHisto:
        smearedHisto.append(np.random.poisson(aBin,1)[0])
    return smearedHisto

<h3>3. Profile LikelyHood defininition</h3>